In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from datetime import datetime
from ema_workbench import load_results

In [2]:
experiment_datetime = "20230611T1944"
results = load_results('./output/' + experiment_datetime + '_results__all_generated')
experiments, outcomes = results
experiments_df = pd.DataFrame(experiments)
outcomes_df = pd.DataFrame(outcomes)

In [3]:
outcomes_df

,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs
0,0.0,0.0,4.424988e+05,0.000051,0.0,0.0,0.000000e+00,0.000000,1.705168e+09
1,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,7.189113e+07,0.007621,1.705168e+09
2,0.0,0.0,6.357918e+06,0.000553,0.0,0.0,1.441078e+08,0.011725,1.705168e+09
3,0.0,0.0,4.867098e+06,0.000545,0.0,0.0,5.576078e+07,0.005936,1.705168e+09
4,0.0,0.0,9.677865e+06,0.000871,0.0,0.0,7.584626e+06,0.000630,1.705168e+09
...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1.178841e+08,0.013715,1.333423e+09
996,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,4.956028e+06,0.000823,1.333423e+09
997,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0.000000e+00,0.000000,1.333423e+09
998,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,3.118616e+06,0.000334,1.333423e+09


## Simple Analysis (Fixed Policies)
Here, we look for statistics about the outcomes of the experiments, separated by policy.

In [8]:
experiments_df = experiments_df.reset_index(names='Run ID')
outcomes_df = outcomes_df.reset_index(names='Run ID')

In [9]:
experiments_df.head()

,Run ID,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,...,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model,Policy
0,0,7,334.222213,10.0,0.798012,205.379941,1.0,0.504147,104.056049,1.0,...,0,4,3,8,2,4,10,0,dikesnet,4
1,1,124,327.089631,1.5,0.882325,143.027126,1.0,0.710557,277.956533,10.0,...,0,4,3,8,2,4,11,0,dikesnet,4
2,2,43,260.244096,1.5,0.592804,305.267854,1.0,0.261648,305.269720,1.5,...,0,4,3,8,2,4,12,0,dikesnet,4
3,3,102,183.624859,10.0,0.612365,222.199248,1.5,0.217695,60.852671,1.0,...,0,4,3,8,2,4,13,0,dikesnet,4
4,4,26,51.404212,1.0,0.808813,182.426007,1.5,0.184892,78.375963,1.5,...,0,4,3,8,2,4,14,0,dikesnet,4


In [10]:
# Tag which policy each row belongs to
experiments_df['Policy'] = ''
for idx, row in experiments_df.iterrows():
    if row['0_RfR 0'] == 1:
        if row['A.4_DikeIncrease 0'] == 5:
            experiments_df.at[idx, 'Policy'] = 5
        else:
            experiments_df.at[idx, 'Policy'] = 1
    elif row['3_RfR 0'] == 1:
        experiments_df.at[idx, 'Policy'] = 2
    elif row['A.4_DikeIncrease 0'] == 5:
        experiments_df.at[idx, 'Policy'] = 3
    else:
        experiments_df.at[idx, 'Policy'] = 4

In [11]:
policies_df = experiments_df[['Run ID', 'Policy']]
policies_df.head()

,Run ID,Policy
0,0,4
1,1,4
2,2,4
3,3,4
4,4,4


In [12]:
outcomes_df = pd.merge(outcomes_df, policies_df, on='Run ID')
outcomes_df.head()

,Run ID,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs,Policy
0,0,0.0,0.0,4.424988e+05,0.000051,0.0,0.0,0.000000e+00,0.000000,1.705168e+09,4
1,1,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,7.189113e+07,0.007621,1.705168e+09,4
2,2,0.0,0.0,6.357918e+06,0.000553,0.0,0.0,1.441078e+08,0.011725,1.705168e+09,4
3,3,0.0,0.0,4.867098e+06,0.000545,0.0,0.0,5.576078e+07,0.005936,1.705168e+09,4
4,4,0.0,0.0,9.677865e+06,0.000871,0.0,0.0,7.584626e+06,0.000630,1.705168e+09,4


In [13]:
means = outcomes_df.groupby('Policy').mean()
means.head()

,Run ID,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs
Policy,,,,,,,,,,
1,449.500000,0.000000e+00,0.000000,1.133389e+07,0.009263,6.906931e+05,0.000235,1.077891e+06,0.000362,1.985586e+09
2,399.500000,1.710838e+07,0.001123,3.370944e+05,0.000052,2.223454e+06,0.000122,3.551899e+06,0.000391,1.886326e+09
4,516.166667,4.322943e+06,0.000310,1.431445e+07,0.001959,1.201852e+06,0.000085,3.475388e+07,0.004134,1.410044e+09
5,849.500000,0.000000e+00,0.000000,1.277450e+05,0.000165,4.294669e+05,0.000208,1.862767e+06,0.001697,1.599070e+09


In [14]:
means['Total Infrastructure Costs']

Policy
1    1.985586e+09
2    1.886326e+09
4    1.410044e+09
5    1.599070e+09
Name: Total Infrastructure Costs, dtype: float64